# Finetuning demo for Whisper
codes adapted from Whisper community [here](https://colab.research.google.com/drive/1P4ClLkPmfsaKn2tBbRp0nVjGMRKR-EWz) and [here](https://huggingface.co/blog/fine-tune-whisper#prepare-environment)

Whisper_base finetuned on Chinese only.

**Content:**

- Preparation
    - Prepare environment
    - Prepare commonvoice/fleurs dataset
    - Prepare Whisper pretrained processor
    - Preprocess dataset with Whisper encoder (saved and can be reloaded)
    - Prepare evaluation metrics (**Start from here with pre-loaded dataset and whisper processor**)
- Finetune
    - Load pretrained Whisper for conditional generation
    - Train Whisper on prepared dataset


[This thread](https://discuss.huggingface.co/t/seq2seqtrainer-enabled-must-be-a-bool-got-nonetype/25680/11) I made on HuggingFace forum may be helpful is any issue happens.

## Prepare Env

In [1]:
!pwd
!conda env list
# import warnings
# warnings.filterwarnings('ignore')

/home/sivan/asr/whisper_finetune
# conda environments:
#
base                     /opt/conda
pytorch_env           *  /opt/conda/envs/pytorch_env



In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Nov  6 23:57:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 495.46       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

Error: must run as root
Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [4]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-848rkjj1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-848rkjj1
  Resolved https://github.com/huggingface/transformers to commit 504db92e7da010070c36e185332420a1d52c12b2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /home/sivan/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


## Plan A: Load dataset from Common Voice

downloading zh-CN set took about 20 mins stored in /home/sivan/.cache/huggingface/datasets

In [2]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", split="test", use_auth_token=True)

print(common_voice)

Found cached dataset common_voice_11_0 (/home/sivan/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/zh-CN/11.0.0/8975395f1d50a6b61f707acd3416761702d3b25412f5fb1004e1db51fe7c304a)
Found cached dataset common_voice_11_0 (/home/sivan/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/zh-CN/11.0.0/8975395f1d50a6b61f707acd3416761702d3b25412f5fb1004e1db51fe7c304a)


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 39637
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 10581
    })
})


In [3]:
# keep only the audio and transcript
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 39637
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 10581
    })
})


## Plan B: try loading fluers instead
taking about 2 mins

In [30]:
from datasets import load_dataset
fleurs_ch = load_dataset("google/fleurs", "cmn_hans_cn")
print(fleurs_ch)

Found cached dataset fleurs (/home/sivan/.cache/huggingface/datasets/google___fleurs/cmn_hans_cn/2.0.0/aabb39fb29739c495517ac904e2886819b6e344702f0a5b5283cb178b087c94a)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 3246
    })
    validation: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 409
    })
    test: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 945
    })
})


In [31]:
print(fleurs_ch["train"][0])

{'id': 1368, 'num_samples': 132480, 'path': '/home/sivan/.cache/huggingface/datasets/downloads/extracted/0453709a21b236bdd65487aa8a2d41cafb9515ca24965f9608ef0983230ce835/cmn_hans_cn/audio/train/5551535928909193992.wav', 'audio': {'path': '5551535928909193992.wav', 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00197393,
       -0.00212777, -0.00208348]), 'sampling_rate': 16000}, 'transcription': '如 果 他 们 使 用 航 空 的 方 式 运 输 货 物 在 某 些 航 线 上 可 能 要 花 几 天 的 时 间 才 能 卸 货 和 通 关', 'raw_transcription': '如果他们使用航空的方式运输货物，在某些航线上，可能要花几天的时间才能卸货和通关。', 'gender': 1, 'lang_id': 13, 'language': 'Mandarin Chinese', 'lang_group_id': 6}


In [32]:
fleurs_ch = fleurs_ch.remove_columns(['id', 'num_samples', 'path','gender', 'lang_id', 'language', 'lang_group_id'])
print(fleurs_ch)

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription', 'raw_transcription'],
        num_rows: 3246
    })
    validation: Dataset({
        features: ['audio', 'transcription', 'raw_transcription'],
        num_rows: 409
    })
    test: Dataset({
        features: ['audio', 'transcription', 'raw_transcription'],
        num_rows: 945
    })
})


## Load Whisper

In [2]:
# feature extractor
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

2022-11-11 15:59:04.985265: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-11 15:59:05.178473: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-11 15:59:05.720422: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-11-11 15:59:05.720513: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

In [3]:
# tokenizer
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="Chinese", task="transcribe")

In [4]:
# processor
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="Chinese", task="transcribe")

## Prepare Data

In [8]:
# from datasets import Audio
# resampling from 48kHz to 16kHz
# common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [40]:
print(fleurs_ch["train"][0])

{'audio': {'path': '5551535928909193992.wav', 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00197393,
       -0.00212777, -0.00208348]), 'sampling_rate': 16000}, 'transcription': '如 果 他 们 使 用 航 空 的 方 式 运 输 货 物 在 某 些 航 线 上 可 能 要 花 几 天 的 时 间 才 能 卸 货 和 通 关', 'raw_transcription': '如果他们使用航空的方式运输货物，在某些航线上，可能要花几天的时间才能卸货和通关。'}


In [6]:
# batch prepare
def prepare_dataset(batch):
    # load resampled audio data
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["transcription"]).input_ids # make sure to encode the transcription column as ground truth
    return batch

In [21]:
# Will take ~40 mins on 16 processes for common voice (and always crash OOM)
# Fleurs takes 3/0.5/1 min for 812/102/237 splits for 4 processes
fleurs_ch = fleurs_ch.map(prepare_dataset, remove_columns=fleurs_ch.column_names["train"], num_proc=4)

#0:   0%|          | 0/812 [00:00<?, ?ex/s]

#1:   0%|          | 0/812 [00:00<?, ?ex/s]

#3:   0%|          | 0/811 [00:00<?, ?ex/s]

#2:   0%|          | 0/811 [00:00<?, ?ex/s]

#0:   0%|          | 0/103 [00:00<?, ?ex/s]

...

#3:   0%|          | 0/102 [00:00<?, ?ex/s]

#0:   0%|          | 0/237 [00:00<?, ?ex/s]

#1:   0%|          | 0/236 [00:00<?, ?ex/s]

#3:   0%|          | 0/236 [00:00<?, ?ex/s]

#2:   0%|          | 0/236 [00:00<?, ?ex/s]

In [22]:
print(fleurs_ch)

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 3246
    })
    validation: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 409
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 945
    })
})


In [24]:
# save processed data to bucket
fleurs_ch.save_to_disk('/home/sivan/datasets/fl_ch_features2')

In [25]:
# uploading takes ~30 secs for 4G
!gsutil -m cp -n -r /home/sivan/datasets/fl_ch_features gs://capstone_datasets/fleurs/preprocess/fl_ch_features

Copying file:///home/sivan/datasets/fl_ch_features/dataset_dict.json [Content-Type=application/json]...
Copying file:///home/sivan/datasets/fl_ch_features/test/state.json [Content-Type=application/json]...
Copying file:///home/sivan/datasets/fl_ch_features/test/dataset.arrow [Content-Type=application/octet-stream]...
Copying file:///home/sivan/datasets/fl_ch_features/test/dataset_info.json [Content-Type=application/json]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled 

## Load data from disk

In [5]:
from datasets import load_dataset, load_from_disk
fleurs_ch = load_from_disk('/home/sivan/datasets/fl_ch_features')
print(fleurs_ch)

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 3246
    })
    validation: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 409
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 945
    })
})


In [6]:
# data collator
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [7]:
# initialize collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

## Prepare eval

In [8]:
import evaluate

metric = evaluate.load("wer")

In [9]:
import sys
sys.path.insert(0, '/home/sivan/asr/whisper')
from utils import custom_normalizer
def compute_metrics(pred):
    pred_ids = pred.predictions # ids from model.generate
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # normalizer
    # pred_str = custom_normalizer(pred_str, "zh")
    pred_str = [custom_normalizer(str(input_str), "zh") for input_str in pred_str]
    label_str = [custom_normalizer(str(input_str), "zh") for input_str in label_str]

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    print(pred_str[:5], label_str[0:5])
    return {"wer": wer}

# Finetuning

### Load pretrained Whisper

In [10]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

In [11]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.use_cache = False

### Training config

In [12]:
# reduce per_device_train_batch_size if cuda OOM
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/home/sivan/whisper_base_fl_ch",
    per_device_train_batch_size=16, # originally 16
    gradient_accumulation_steps=1,  # originally 1, increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=False, # original True
    group_by_length=False, # set true if length is specified in dataset
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False, # set true to push trained model to HF
    disable_tqdm=False, # set false to see progress bar
)

In [13]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=fleurs_ch["train"],
    eval_dataset=fleurs_ch["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
%%time
trainer.train()

# Choose best model on validation set

In [12]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="/home/sivan/whisper_base_fl_ch/validations",
    do_train=False,
    do_eval=True,
    per_device_eval_batch_size=32, # 32 for 8000+MiB
    predict_with_generate=True,
    generation_max_length=225,
    save_strategy="no",
    report_to=["tensorboard"],
    push_to_hub=False,
    disable_tqdm=False,
)

from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="Chinese", task="transcribe")

In [13]:
%%time
# eval on all checkpoints (~7 mins)
from transformers import WhisperForConditionalGeneration
import pandas as pd
import os
results = []
for step in [1000,2000,3000,4000]:
    model = WhisperForConditionalGeneration.from_pretrained("/home/sivan/whisper_base_fl_ch/checkpoint-{}".format(step))
    print("model loaded from checkpoint ", step)
    trainer = Seq2SeqTrainer(
        args=training_args,
        model=model,
        eval_dataset=fleurs_ch["validation"],
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        tokenizer=processor.feature_extractor,
    )
    results.append(trainer.evaluate())

# save reuslts
results = pd.DataFrame(results)
results.to_csv(os.path.join(os.getcwd(), 'wer_fl_ch.csv'))

model loaded from checkpoint  1000


***** Running Evaluation *****
  Num examples = 409
  Batch size = 32


loading configuration file /home/sivan/whisper_base_fl_ch/checkpoint-2000/config.json
Model config WhisperConfig {
  "_name_or_path": "openai/whisper-base",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "WhisperForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "begin_suppress_tokens": [
    220,
    50257
  ],
  "bos_token_id": 50257,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 50258,
  "dropout": 0.0,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 50257,
  "forced_decoder_ids": null,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_length": 448,
  "max_source_positions": 1500,
  "max_target_positions": 448,
  "model_type": "whisper",
  "num_hidden_layers": 6,
  "num_mel_bins": 80,
  "pad_token_id": 50257,
  "scale_embedding": false,
  "

['加 南 没 有 大 胜 利 所 以 木 材 非 常 昂 贵 ', '由 于 一 天 只 颁 发 18 块 奖 牌 许 多 国 家 的 运 动 员 未 能 登 上 邻 较 台 ', '公 园 站 第 19 500 平 方 公 里 分 为 14 个 不 同 的 生 态 区 为 不 同 的 野 生 动 植 物 提 供 支 持 ', '请 及 与 此 地 应 得 的 尊 严 装 重 和 尊 重 不 要 用 大 屠 杀 混 纳 粹 开 玩 笑 ', '莫 堵 第 一 次 世 界 大 战 中 的 私 女 残 暴 后 各 国 都 可 望 避 免 重 打 负 责 ', '是 为 国 家 安 全 国 问 塔 萨 建 筑 卡 特 徒 利 诸 多 国 际 事 物 例 如 1978 年 接 触 待 维 英 协 议 在 20 世 纪 70 年 来 末 实 现 美 中 关 系 正 常 化 1979 年 导 致 伊 朗 人 之 威 胁 的 伊 朗 革 命 1979 年 苏 联 入 侵 阿 富 汗 r u s t u s a r f o h a n n e s o n ', '还 有 设 会 和 政 治 影 响 例 如 杜 良 单 位 的 使 用 从 钳 次 到 共 和 质 的 转 变 民 族 主 义 相 信 国 家 不 属 于 危 机 的 统 治 者 而 是 属 于 人 民 ', '在 这 个 名 人 的 村 中 漫 步 半 小 时 是 值 得 的 ', '在 宽 国 的 利 益 那 岛 b e r l í b a y 厢 的 建 筑 和 线 的 话 的 构 物 中 心 中 间 散 布 着 传 统 的 红 马 无 定 十 八 世 纪 的 市 场 以 及 古 老 的 清 晨 司 和 教 堂 不 过 这 所 曾 是 比 传 统 的 土 耳 其 更 有 地 重 海 欧 洲 的 氛 围 ', '照 堂 目 的 领 一 先 焚 目 上 方 板 这 有 趣 的 大 理 石 隔 子 雕 所 '] ['迦 南 没 有 大 森 林 所 以 木 材 非 常 昂 贵 ', '由 于 一 天 只 颁 发 十 八 块 奖 牌 许 多 国 家 的 运 动 员 未 能 登 上 领 奖 台 ', '公 园 占 地 19500 平 方 公 里 分 为 14 个 不 同 的 生 态 区 为 不 同 的 野 生 动 植 物 提 

All model checkpoint weights were used when initializing WhisperForConditionalGeneration.

All the weights of WhisperForConditionalGeneration were initialized from the model checkpoint at /home/sivan/whisper_base_fl_ch/checkpoint-2000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use WhisperForConditionalGeneration for predictions without further training.
***** Running Evaluation *****
  Num examples = 409
  Batch size = 32


...

loading configuration file /home/sivan/whisper_base_fl_ch/checkpoint-4000/config.json
Model config WhisperConfig {
  "_name_or_path": "openai/whisper-base",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "WhisperForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "begin_suppress_tokens": [
    220,
    50257
  ],
  "bos_token_id": 50257,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 50258,
  "dropout": 0.0,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 50257,
  "forced_decoder_ids": null,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_length": 448,
  "max_source_positions": 1500,
  "max_target_positions": 448,
  "model_type": "whisper",
  "num_hidden_layers": 6,
  "num_mel_bins": 80,
  "pad_token_id": 50257,
  "scale_embedding": false,
  "

['加 拿 人 没 有 大 胜 利 所 以 牧 材 非 常 昂 贵 ', '由 于 一 天 只 班 发 18 块 奖 牌 许 多 国 家 的 运 动 员 未 能 登 上 邻 较 台 ', '公 园 站 第 19 500 平 方 公 里 分 为 14 个 不 同 的 生 态 区 为 不 同 的 野 生 动 植 物 提 供 支 持 ', '请 及 与 此 地 英 德 的 尊 严 装 重 和 尊 重 不 要 用 大 屠 杀 混 纳 粹 开 玩 笑 ', '莫 堵 第 一 次 世 界 大 战 中 的 私 女 残 暴 后 各 国 都 可 望 避 免 重 岛 负 责 ', '是 美 国 家 安 全 国 问 塔 克 森 写 住 卡 特 t r e n t y 出 多 国 际 事 物 例 如 1978 年 接 触 带 位 置 写 议 在 20 世 纪 70 年 来 没 实 现 美 中 关 系 正 常 化 1979 年 导 致 伊 朗 人 之 危 机 的 伊 朗 革 命 1979 年 苏 联 入 侵 阿 富 汗 ', '还 有 设 会 和 政 治 影 响 例 如 杜 良 单 位 的 使 用 从 川 次 到 共 和 兹 的 转 变 民 族 主 义 相 信 国 家 不 属 于 危 机 的 统 治 者 而 是 属 于 人 民 ', '在 这 个 名 人 的 村 中 曼 布 半 小 时 是 值 得 的 ', '在 宽 过 的 利 因 那 道 柏 里 外 墙 的 建 筑 和 线 的 话 的 构 物 中 心 中 间 散 布 着 传 统 的 红 马 无 定 18 世 纪 的 市 场 以 及 古 老 的 清 晨 司 和 教 堂 不 过 这 座 城 是 比 传 统 的 土 耳 其 更 有 地 中 海 欧 洲 的 氛 围 ', '照 堂 目 的 领 一 先 房 目 上 方 摆 着 有 趣 的 大 礼 石 格 子 雕 所 '] ['迦 南 没 有 大 森 林 所 以 木 材 非 常 昂 贵 ', '由 于 一 天 只 颁 发 十 八 块 奖 牌 许 多 国 家 的 运 动 员 未 能 登 上 领 奖 台 ', '公 园 占 地 19500 平 方 公 里 分 为 14 个 不 同 的 生 态 区 为 不 同 的 野 生 动 植 物 提 供 支 持 ', '请 给 予 此 地 应 得 的 尊 严 庄 严 和

All model checkpoint weights were used when initializing WhisperForConditionalGeneration.

All the weights of WhisperForConditionalGeneration were initialized from the model checkpoint at /home/sivan/whisper_base_fl_ch/checkpoint-4000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use WhisperForConditionalGeneration for predictions without further training.
***** Running Evaluation *****
  Num examples = 409
  Batch size = 32


model loaded from checkpoint  4000


['加 拿 人 没 有 大 胜 利 所 以 牧 材 非 常 昂 贵 ', '由 于 一 天 只 班 发 18 块 奖 牌 许 多 国 家 的 运 动 员 未 能 登 上 邻 较 台 ', '公 园 站 第 19 500 平 方 公 里 分 为 14 个 不 同 的 生 态 区 为 不 同 的 野 生 动 植 物 提 供 支 持 ', '请 及 与 此 地 英 德 的 尊 严 装 重 和 尊 重 不 要 用 大 屠 杀 混 纳 磺 开 玩 笑 ', '莫 堵 第 一 次 世 界 大 战 中 的 私 女 残 暴 后 各 国 都 可 望 避 免 重 岛 负 责 ', '是 美 国 家 安 全 国 问 塔 克 森 写 住 卡 特 t r e n t y 出 多 国 际 事 物 例 如 1978 年 接 触 带 位 置 写 议 在 20 世 纪 70 年 来 没 实 现 美 中 关 系 正 常 化 1979 年 导 致 伊 朗 人 直 播 机 的 伊 朗 革 命 1979 年 苏 联 入 侵 阿 富 汗 ', '还 有 设 会 和 政 治 影 响 例 如 杜 良 单 位 的 使 用 从 川 次 到 共 和 兹 的 转 变 民 族 主 义 相 信 国 家 不 属 于 危 机 的 统 治 者 而 是 属 于 人 民 ', '在 这 个 名 人 的 村 中 曼 布 半 小 时 是 值 得 的 ', '在 宽 过 的 利 因 那 道 柏 里 外 墙 的 建 筑 和 线 的 话 的 构 物 中 心 中 间 散 布 着 传 统 的 红 马 无 定 18 世 纪 的 市 场 以 及 古 老 的 清 晨 四 和 教 党 不 过 这 所 曾 是 比 传 统 的 土 耳 其 更 有 地 中 海 欧 洲 的 氛 围 ', '照 堂 目 的 领 一 先 房 目 上 方 摆 着 有 趣 的 大 礼 石 格 子 雕 所 '] ['迦 南 没 有 大 森 林 所 以 木 材 非 常 昂 贵 ', '由 于 一 天 只 颁 发 十 八 块 奖 牌 许 多 国 家 的 运 动 员 未 能 登 上 领 奖 台 ', '公 园 占 地 19500 平 方 公 里 分 为 14 个 不 同 的 生 态 区 为 不 同 的 野 生 动 植 物 提 供 支 持 ', '请 给 予 此 地 应 得 的 尊 严 庄 严 和

In [14]:
import pandas as pd
results = pd.read_csv(os.path.join(os.getcwd(), 'wer_fl_ch.csv'))
print(results)

   Unnamed: 0  eval_loss   eval_wer  eval_runtime  eval_samples_per_second  \
0           0   0.485646  18.331450      102.6121                    3.986   
1           1   0.556010  18.451540      101.7179                    4.021   
2           2   0.584909  18.762362      102.8929                    3.975   
3           3   0.596293  18.783555      102.6578                    3.984   

   eval_steps_per_second  
0                  0.127  
1                  0.128  
2                  0.126  
3                  0.127  


In [15]:
predict_results  = trainer.predict(fleurs_ch["test"], metric_key_prefix="test")
metrics = predict_results.metrics
trainer.log_metrics("test", metrics)
trainer.save_metrics("test", metrics)

***** Running Prediction *****
  Num examples = 945
  Batch size = 32


['1940 年 8 月 15 日 蒙 军 攻 入 法 国 南 部 这 次 军 官 被 称 为 龙 齐 冰 行 动 ', '该 群 群 岛 位 于 南 极 半 岛 一 倍 120 公 里 处 最 大 的 岛 屿 是 桥 制 国 王 岛 这 里 是 梵 新 村 的 定 居 点 ', '捐 赛 这 些 颜 色 将 根 据 化 学 物 质 的 三 件 性 发 生 变 化 ', '海 地 正 与 民 主 研 究 所 仅 用 的 独 立 研 究 表 明 是 尼 博 尔 的 联 合 国 维 和 部 队 在 不 知 情 的 情 况 下 将 这 种 疾 病 带 到 了 海 地 ', '劳 不 使 全 能 运 动 远 它 会 判 划 虽 然 判 划 能 力 不 是 很 强 游 泳 跳 远 拉 利 能 达 到 人 类 大 力 士 的 无 贝 ', '在 这 份 报 告 的 78 项 建 议 中 第 一 条 建 议 是 在 今 年 年 底 之 前 才 取 心 的 外 交 行 动 目 的 在 于 保 护 伊 拉 克 的 编 辑 免 受 敌 对 势 力 的 迁 害 并 与 英 国 重 建 外 交 关 系 ', '你 可 以 乘 坐 波 达 波 达 出 租 的 摩 托 车 游 览 格 马 短 托 车 乘 的 正 常 当 地 价 格 是 500 国 法 兰 ', '区 域 性 和 继 续 性 的 恶 劣 天 气 现 象 包 括 暴 风 雪 雪 暴 沙 暴 和 沙 城 暴 等 ', '最 终 有 许 多 小 型 的 毛 克 动 包 括 鲨 养 的 虫 毛 以 昆 虫 液 尺 动 物 细 艺 和 鸟 类 等 数 量 多 得 多 的 小 骨 针 列 物 为 事 ', '每 个 人 都 是 社 会 的 一 分 子 都 会 使 用 交 通 系 统 而 几 乎 每 个 人 都 对 交 通 系 统 有 愿 意 演 '] ['1940 年 8 月 15 日 盟 军 攻 入 法 国 南 部 这 次 进 攻 被 称 为 龙 骑 兵 行 动 ', '该 群 岛 位 于 南 极 半 岛 以 北 120 公 里 处 最 大 的 岛 屿 是 乔 治 国 王 岛 这 里 是 繁 星 村 v i l l a l a s e s t r e l l a s 的 定 居 点 ', '卷 心 菜 汁 的 颜 色 将 根 据 化 学 